In [ ]:
%cd ../..

# 11_address_category

#### **WARNING:** This notebook mutates the database. Create a backup of the database before running.

Assign a category to conference addresses.

In [ ]:
import sqlite3

from matplotlib import pyplot as plt

from lectern.analyze import namespaces

In [ ]:
table = namespaces.TABLE_GENERAL_CONFERENCE_ADDRESS
with sqlite3.connect(namespaces.DATABASE_APP_DEFAULT) as con:
  col_infos = con.execute('PRAGMA table_info({});'.format(table))
  cols = [info[1] for info in col_infos]
  cols_to_index = {col: i for i, col in enumerate(cols)}
  rows = list(con.execute('SELECT * FROM {} ORDER BY conference ASC, ordinal ASC;'.format(table)))
print(cols)
print(len(rows))
rows[0]

How many conferences have occurred?

In [ ]:
conferences = [row[cols_to_index['conference']] for row in rows]
len(set(conferences))

## Sustainings

These occur every conference.

We _would_ expect to match the number of sustainings to match the total number of conferences, but in April 2018, both a [solemn assembly](https://churchofjesuschrist.org/study/general-conference/2018/04/solemn-assembly?lang=eng) to sustain the new prophet, Russell M. Nelson, and his Presidency _and_ a [sustaining](https://churchofjesuschrist.org/study/general-conference/2018/04/the-sustaining-of-church-officers?lang=eng) for the other general officers were held.

In [ ]:
sustaining_rows = list()
for row in rows:
  title = row[cols_to_index['title']]
  if (('Sustaining' in title or 'Sustained' in title) and 'Officers' in title) or\
    'Solemn Assembly' in title:
    sustaining_rows.append(row)
    if len(sustaining_rows) == 1:
      continue
    year, month = row[cols_to_index['conference']].split('-')
    last_row = sustaining_rows[-2]
    last_year, last_month = last_row[cols_to_index['conference']].split('-')
    if year == last_year and month == last_month:
      print('Duplicate addresses in same year: {}'.format(year))
      print(last_row)
      print(row)
      print()
    elif int(year) == int(last_year) + 1 and month == last_month:
      print('Missing address after conference: {}-{}'.format(last_year, last_month))
len(sustaining_rows)

## Statistical Reports

These usually occurred once per year, but were discontinued [in 2018](https://churchofjesuschrist.org/study/general-conference/2018/04/the-sustaining-of-church-officers?lang=eng):

> We note that the statistical report which has traditionally been presented during this session of April general conference will now be published on LDS.org immediately following this meeting and will be included in the conference issue of the Church magazines.

This means that the number of statistical reports given should be `2017 - 1970 = 47`.

In [ ]:
statistical_rows = list()
for row in rows:
  title = row[cols_to_index['title']]
  if ('Statistical Report' in title or  # Most years.
      'Annual Report' in title):  # Just 1972.
    year, month = row[cols_to_index['conference']].split('-')
    statistical_rows.append(row)
    if month != '04':
      print('Address given in October:')
      print(row)
      print()
    if len(statistical_rows) == 1:
      continue
    last_row = statistical_rows[-2]
    last_year, last_month = last_row[cols_to_index['conference']].split('-')
    duration = int(year) - int(last_year)
    if duration == 2:
      print('Missing year between {} and {}.'.format(last_year, year))
    elif duration == 0:
      print('Duplicate year {}:'.format(year))
      print(last_row)
      print(row)
      print()
len(statistical_rows)

## Audit Report

These occur every year in April.

In [ ]:
audit_rows = list()
for row in rows:
  title = row[cols_to_index['title']]
  if ('Finance Committee' in title or  # 1974 - 1980.
      'Audit' in title):  # All other years.
    audit_rows.append(row)
    if len(audit_rows) == 1:
      continue
    year, month = row[cols_to_index['conference']].split('-')
    last_row = audit_rows[-2]
    last_year, last_month = last_row[cols_to_index['conference']].split('-')
    duration = int(year) - int(last_year)
    if duration > 1:
      print('Missing year between {} and {}.'.format(last_year, year))
    elif duration == 0:
      print('Duplicate year {}:'.format(year))
      print(last_row)
      print(row)
      print()
len(audit_rows)

## Video

The first one that appears on the Church website as its own address occurred [in October, 2020](https://www.churchofjesuschrist.org/study/general-conference/2020/10/33video).

In [ ]:
video_rows = list()
for row in rows:
  title = row[cols_to_index['title']]
  if (title.startswith('Video:') or
      (title.startswith('Faith in Every Footstep:') and row[cols_to_index['conference']] == '1997-04')):
    video_rows.append(row)
len(video_rows)

## Update

Ensure that each row is assigned to only one category.

In [ ]:
update_rows = [
  *sustaining_rows,
  *statistical_rows,
  *audit_rows,
  *video_rows
]
update_keys = set()
for row in update_rows:
  key = tuple(row[:2])
  if key in update_keys:
    raise ValueError('Duplicate update key: {}'.format(key))
  update_keys.add(key)
print(len(update_keys))
len(update_keys) == len(update_rows)

In [ ]:
def update_category(rows_, category_, db_path_=namespaces.DATABASE_APP_DEFAULT):
  updates = [(category_, *row[:2]) for row in rows_]
  with sqlite3.connect(db_path_) as con:
    con.executemany('UPDATE {} SET category=? WHERE conference=? AND ordinal=?'.format(table), updates)

In [ ]:
update_category(sustaining_rows, 'sustaining')
update_category(statistical_rows, 'statistical')
update_category(audit_rows, 'audit')
update_category(video_rows, 'video')